In [1]:
import numpy as np
import rasterio
import geopandas as gpd
from shapely.geometry import box
from rasterio.mask import mask
import progressbar

In [2]:
# Read the file with rasterio
data_dem = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/DEM_Aletsch_Winter.tif") #AletschAlpsWinter
data_sat = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/SAT_Aletsch_Winter.tif") #AletschAlpsWinter

#data_dem = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/DEM_Bernese_Alps_summer.tif") #BerneseAlpsSummer
#data_sat = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/SAT_Bernese_Alps_summer.tif") #BerneseAlpsSummer

#data_dem = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/DEM_Alps_Summer_3.tif") #Alps3
#data_sat = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/SAT_Alps_Summer_3.tif") #Alps3

#data_dem = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/DEM_Alps_Summer.tif") #Alps1
#data_sat = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/SAT_Alps_Summer.tif") #Alps1

#data_dem = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/DEM_Alps_Summer_2.tif") #Alps2
#data_sat = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/SAT_Alps_Summer_2.tif") #Alps2



#data_dem = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/DEM_Allgaeu_Summer.tif") #AllgaeuSummer
#data_sat = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/SAT_Allgaeu_Summer.tif") #AllgaeuSummer

#data_dem = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/DEM_Bernese_Alps_winter.tif") #Bernese Winter
#data_sat = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/SAT_Bernese_Alps_Winter.tif") #Bernese Sinter
                

In [3]:
def generate_random_bbox(data_sat, nx=5000, ny=5000):
    """Generate random bbox within domain size of images
    
    nx = int: cells in x-direction
    ny = int: cells in y-direction
    """
    # set bounds from sat image for now:
    global_min_x = data_sat.bounds[0]
    global_min_y = data_sat.bounds[1]
    global_max_x = data_sat.bounds[2]
    global_max_y = data_sat.bounds[3]

    # get random location
    min_x = np.random.randint(global_min_x, global_max_x-nx)
    max_x = min_x + nx
    min_y = np.random.randint(global_min_y, global_max_y-ny)
    max_y = min_y + ny

    
    bbox = box(min_x, min_y, max_x, max_y)

    geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0])
    
    return geo

def extract_images(data_sat, data_dem, geo):
    """Extract subimages from sat and dem
    """

    # Get the geometry coordinates
    coords = getFeatures(geo)

    # Clip the raster with the polygon
    out_img_sat, out_transform_sat = mask(data_sat, shapes=coords, crop=True)
    out_img_dem, out_transform_dtm = mask(data_dem, shapes=coords, crop=True)
    
    return out_img_sat, out_img_dem

def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [ ]:
geo = generate_random_bbox(data_sat, nx=6200, ny=4700)

In [ ]:
out_img_sat, out_img_dem = extract_images(data_sat, data_dem, geo)

In [ ]:
out_img_sat.shape, out_img_dem.shape

In [ ]:
data_dem.shape


In [4]:
def reshape_and_reorder(out_img_sat, out_img_dem):
    # remove edges
    out_img_sat = out_img_sat[:,2:-2,2:-2]
    out_img_dem = out_img_dem[:,2:-2,2:-2]
    # reorder axes
    out_img_sat_reshape = np.moveaxis(out_img_sat, 0, -1)
    out_img_dem_reshape = np.moveaxis(out_img_dem, 0, -1)
    
    return out_img_sat_reshape, out_img_dem_reshape

In [5]:
n = 200050

for i in progressbar.progressbar(range(n)):
    geo = generate_random_bbox(data_sat, nx=6200, ny=4700)
    out_img_sat, out_img_dem = extract_images(data_sat, data_dem, geo)
    out_img_sat_reshape, out_img_dem_reshape = reshape_and_reorder(out_img_sat, out_img_dem)
    # save in numpy binary
    #np.save("C:/Users/robin/Google Drive/BA/Wellmann SatImg/OutputImagesSAT/out_img_sat_%04d.npy" % i, out_img_sat_reshape)
    #np.save("C:/Users/robin/Google Drive/BA/Wellmann SatImg/OutputImagesDEM/out_img_dem_%04d.npy" % i, out_img_dem_reshape)
    #np.save("D:/BA_/output_processed/Sat/out_img_sat_%06d.npy" % i, out_img_sat_reshape) #D:\
    #np.save("D:/BA_/output_processed/DEM/out_img_dem_%06d.npy" % i, out_img_dem_reshape) #D:\
    np.save("F:/BA100k/SAT/out_img_sat_%06d.npy" % i, out_img_sat_reshape)
    np.save("F:/BA100k/DEM/out_img_dem%06d.npy" % i, out_img_dem_reshape)

100% (200050 of 200050) |################| Elapsed Time: 1:34:01 Time:  1:34:01


In [ ]:
out_img_sat_reshape.shape, out_img_dem_reshape.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(out_img_dem_reshape[:,:,0])

In [ ]:
img_SAT = np.fromfile("C:/Users/robin/Desktop/Wellmann SatImg/OutputImagesSAT/out_img_sat_0000.npy")

In [ ]:
img_DEM = np.fromfile("C:/Users/robin/Desktop/Wellmann SatImg/OutputImagesDEM/out_img_dem_0000.npy")

In [ ]:
img_SAT_reshaped, img_DEM_reshaped = reshape_and_reorder(img_SAT, img_DEM)